# ResNet-Based mmFace

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels)
        )
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)

        return out
    
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)
    
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes)
            )
        layers = [block(self.inplanes, planes, stride, downsample)] + [block(planes, planes) for _ in range(blocks-1)]
        self.inplanes = planes

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

## Loading Dataset and Model

In [69]:
import numpy as np
from glob import glob
import json
from utils import get_crd_data

def load_dataset(path):
    radar_jsons = glob(f"{path}/*/*.json")
    data = []
    for i, file in enumerate(tqdm(radar_jsons)):
        if "100" in file:
            continue
        if i == 2:
            break
        # TODO: BATCH THESE AND ADD LABELS SO DATASET LOOKS LIKE (3750 BATCHES, (16 (32, 16, 3) CRD FRAMES, 16 LABELS)) 
        # 16 Subjects x 15 Scenarios x 250 Frames = 60,000 / 16 = 3,750 Batches
        with open(file, 'r') as f:
            data.append(np.einsum("fcrd->frdc", get_crd_data(json.load(f), num_chirps_per_burst=16)[:250]))
        
    return np.array(data)

def load_model(model, optimiser):
    epoch, loss = 0, None
    try:
        checkpoint = torch.load("model/mmFace.pt")
        model.load_state_dict(checkpoint["model_state_dict"])
        optimiser.load_state_dict(checkpoint["optimiser_state_dict"])
        epoch = checkpoint["epoch"]
        loss = checkpoint["loss"]
    except Exception as ex:
        print(ex)
    
    return epoch, loss

dataset = load_dataset("../../Soli/soli_realsense/data")
print(dataset.shape)

  1%|          | 2/245 [00:01<03:38,  1.11it/s]

(2, 250, 32, 16, 3)


## Hyperparameters + Loss + Optimiser

In [ ]:
num_classes = 10
num_epochs = 20
batch_size = 16
learning_rate = 0.01

model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)
# Loss + Optimiser
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.001, momentum=0.9)

cur_epoch, cur_loss = load_model(model, optimiser)

## Training
- Load training data in ***batches*** for every epoch, moving to `device`
  - `train_loader` = `[([data*], [labels*])*]`
- `model(data)` to predict label, then calculate loss between predictions and ground truth labels using `criterion(preds, labels)`
- Backpropagate to learn with `loss.backward()`, and update weights with `optimiser.step()`. Gradients must be reset to 0 after every update with `optimiser.zero_grad()` otherwise gradients will accumulate (default PyTorch).
- After every epoch, test model on validation set but can turn off gradients for faster evaluation using `with torch.no_grad()`.

In [ ]:
import gc

for epoch in tqdm(range(cur_epoch, num_epochs)):
    for data, labels in tqdm(train):
        data = data.to(device)
        labels = labels.to(device)

        # Forward Pass
        outputs = model(data)
        loss = criterion(outputs, labels)

        # Backward Pass and Optimise
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        del data, labels, outputs
        torch.cuda.empty_cache()
        gc.collect()
    
    torch.save({"epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimiser_state_dict": optimiser.state_dict(),
                "loss": loss},
                "model/mmFace.pt")
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for data, labels in validation:
            data = data.to(device)
            labels = labels.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del data, labels, outputs
        
        print(f"Accuracy of mmFace on Validation: {100*correct/total}%")

## Testing

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data, labels in tqdm(test):
        data = data.to(device)
        labels = labels.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del data, labels, outputs
    
    print(f"Test Accuracy of mmFace: {100*correct/total}")

In [4]:
from tqdm import tqdm

x = [(4, "hello"), (2, 4)]
for p, q in tqdm(x):
    print(p, q)

100%|██████████| 2/2 [00:00<?, ?it/s]

4 hello
2 4
